<a href="https://colab.research.google.com/github/MPrazzoli/corpsol/blob/main/Script_buono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install QuantLib

     |████████████████████████████████| 20.1MB 1.4MB/s 


In [4]:
import os
import math
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date

**Inserimento degli Input**

In [5]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EUR3M'
except:
  underlying = 'EUR3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

Inserire il sottostante: 
Inserire il piano di ammortamento: 
Inserire il nozionale: 
Inserire il tasso annuo: 
Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): 
Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: 
Inserire il periodo che intercorre tra le rate in numero di mesi: 
Inserire la convenzione:
Inserire la regola per il calcolo delle date:Backward


In [100]:
calendar = TARGET()
start_date = calendar.advance(ref_date,2,Days)
if maturity_label[-1] == 'M':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
elif maturity_label[-1] == 'Y':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
Tenor = Period(tenor,Months)
endofmonth = False
terminationDateConvention = business_convention
df=pd.DataFrame({'startdate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[:-1],'enddate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[1:]})


In [28]:
if maturity_label[-1] == 'M':
  rate_totali = int(12/tenor)
elif maturity_label[-1] == 'Y':
  rate_totali = int((12/tenor)*int(maturity_label.rsplit(maturity_label[-1], 1)[0]))

In [ ]:
if (amortization_type== 'Francese'):
  #Calcolo della rata 
  t = (1+ amortization_rate/(12/tenor))**rate_totali
  rata=round(notional*t*(amortization_rate/(12/tenor))/(t-1),2)
  Residuo=notional
  Estinto=0
  tot_I=0
  plan=np.array
  plan=[[0,0,0,0,0,Residuo]]
  rate=range(0,rate_totali)
  for count in rate:
    Int_quota=round(Residuo*(amortization_rate*(tenor/12)),2)
    Cap_quota=(rata-Int_quota)
    tot_I=(tot_I+Int_quota)
    Residuo=(Residuo-Cap_quota)
    Estinto=(Estinto+Cap_quota)
    Tot=(Int_quota+Cap_quota)
    plan.append([Int_quota,Cap_quota,Tot,tot_I,Estinto,Residuo])
  index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_fra = pd.DataFrame(plan)
  df_fra.columns=index
  df1_fra=pd.concat([df,df_fra],axis=1)
  df2_fra=pd.concat([df,df_fra['Debito Residuo']],axis=1)[:-1]
  print(df2_fra)
  
elif (amortization_type == 'Italiano'):
  Residuo_ita=notional
  tot_I_ita=0
  Estinto_ita=0
  Tot_ita=0
  plan_ita=np.array
  plan_ita=[[0,0,0,0,0,Residuo_ita]]
  rate_ita=range(0,rate_totali)
  for count in rate_ita:
    Int_quota_ita=round(Residuo_ita*amortization_rate/(12/tenor),2)
    Cap_quota_ita=round((notional/rate_totali),2)
    tot_I_ita=(tot_I_ita+Int_quota_ita)
    Residuo_ita=(Residuo_ita-Cap_quota_ita)
    Estinto_ita=(Estinto_ita+Cap_quota_ita)
    Tot_ita=(Int_quota_ita+Cap_quota_ita)
    plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])
  index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_ita=pd.DataFrame(plan_ita)
  df_ita.columns=index
  df1_ita=pd.concat([df,df_ita],axis=1)
  df2_ita=pd.concat([df,df_ita['Debito Residuo']],axis=1)
  df2_ita=df2_ita[:-1]
  print(df2_ita)

elif (amortization_type == 'Bullet'):
  Residuo_bullet=notional
  tot_I_bullet=0
  plan_bullet=np.array
  plan_bullet=[[0,0,Residuo_bullet]]
  rate_bullet=range(0,rate_totali)
  for count in rate_bullet:
    Int_quota_bullet=round(Residuo_bullet*amortization_rate/(12/tenor),2)
    tot_I_bullet=(tot_I_bullet+Int_quota_bullet)
    plan_bullet.append([Int_quota_bullet,tot_I_bullet,Residuo_bullet])
  index=['Quota Interesse','Interessi Totali','Debito Residuo']
  df_bullet=pd.DataFrame(plan_bullet)
  df_bullet.columns=index
  df1_bullet=pd.concat([df,df_bullet],axis=1)
  df2_bullet=pd.concat([df,df_bullet['Debito Residuo']],axis=1)
  df2_bullet=df2_bullet[:-1]
  print(df2_bullet)



**Ammortamento Francese**

In [101]:
  #Calcolo della rata 
  t = (1+ amortization_rate/(12/tenor))**rate_totali
  rata=round(notional*t*(amortization_rate/(12/tenor))/(t-1),2)
  Residuo=notional
  Estinto=0
  tot_I=0
  plan=np.array
  plan=[[0,0,0,0,0,Residuo]]

In [102]:
  rate=range(0,rate_totali)
  for count in rate:
    Int_quota=round(Residuo*(amortization_rate*(tenor/12)),2)
    Cap_quota=(rata-Int_quota)
    tot_I=(tot_I+Int_quota)
    Residuo=(Residuo-Cap_quota)
    Estinto=(Estinto+Cap_quota)
    Tot=(Int_quota+Cap_quota)
    plan.append([Int_quota,Cap_quota,Tot,tot_I,Estinto,Residuo])
    index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
df_ita=pd.DataFrame(plan_ita)
df_ita.columns=index
df1_ita=pd.concat([df,df_ita],axis=1)
df2_ita=pd.concat([df,df_ita['Debito Residuo']],axis=1)
df2_ita=df2_ita[:-1]

In [104]:
  index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_fra = pd.DataFrame(plan)
  df_fra.columns=index
  df1_fra=pd.concat([df,df_fra],axis=1)
  df2_fra=pd.concat([df,df_fra['Debito Residuo']],axis=1)[:-1]


**Ammortamento** **Italiano**

In [108]:
Residuo_ita=notional
tot_I_ita=0
Estinto_ita=0
Tot_ita=0
plan_ita=np.array
plan_ita=[[0,0,0,0,0,Residuo_ita]]
rate_ita=range(0,rate_totali)
for count in rate_ita:
  Int_quota_ita=round(Residuo_ita*amortization_rate/(12/tenor),2)
  Cap_quota_ita=round((notional/rate_totali),2)
  tot_I_ita=(tot_I_ita+Int_quota_ita)
  Residuo_ita=(Residuo_ita-Cap_quota_ita)
  Estinto_ita=(Estinto_ita+Cap_quota_ita)
  Tot_ita=(Int_quota_ita+Cap_quota_ita)
  plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])
  index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
df_ita=pd.DataFrame(plan_ita)
df_ita.columns=index
df1_ita=pd.concat([df,df_ita],axis=1)
df2_ita=pd.concat([df,df_ita['Debito Residuo']],axis=1)
df2_ita=df2_ita[:-1]

In [109]:
rate_ita=range(0,rate_totali)
for count in rate_ita:
  Int_quota_ita=round(Residuo_ita*amortization_rate/(12/tenor),2)
  Cap_quota_ita=round((notional/rate_totali),2)
  tot_I_ita=(tot_I_ita+Int_quota_ita)
  Residuo_ita=(Residuo_ita-Cap_quota_ita)
  Estinto_ita=(Estinto_ita+Cap_quota_ita)
  Tot_ita=(Int_quota_ita+Cap_quota_ita)
  plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])

In [110]:
index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
df_ita=pd.DataFrame(plan_ita)
df_ita.columns=index
df1_ita=pd.concat([df,df_ita],axis=1)
df2_ita=pd.concat([df,df_ita['Debito Residuo']],axis=1)
df2_ita=df2_ita[:-1]

In [112]:
df2_ita

,startdate,enddate,Debito Residuo
0,"April 27th, 2021","July 27th, 2021",100000.0
1,"July 27th, 2021","October 27th, 2021",95000.0
2,"October 27th, 2021","January 27th, 2022",90000.0
3,"January 27th, 2022","April 27th, 2022",85000.0
4,"April 27th, 2022","July 27th, 2022",80000.0
5,"July 27th, 2022","October 27th, 2022",75000.0
6,"October 27th, 2022","January 27th, 2023",70000.0
7,"January 27th, 2023","April 27th, 2023",65000.0
8,"April 27th, 2023","July 27th, 2023",60000.0
9,"July 27th, 2023","October 27th, 2023",55000.0


**Ammortamento** **Bullet**

In [114]:
Residuo_bullet=notional
tot_I_bullet=0
plan_bullet=np.array
plan_bullet=[[0,0,Residuo_bullet]]
rate_bullet=range(0,rate_totali)
for count in rate_bullet:
  Int_quota_bullet=round(Residuo_bullet*amortization_rate/(12/tenor),2)
  tot_I_bullet=(tot_I_bullet+Int_quota_bullet)
  plan_bullet.append([Int_quota_bullet,tot_I_bullet,Residuo_bullet])
index=['Quota Interesse','Interessi Totali','Debito Residuo']
df_bullet=pd.DataFrame(plan_bullet)
df_bullet.columns=index
df1_bullet=pd.concat([df,df_bullet],axis=1)
df2_bullet=pd.concat([df,df_bullet['Debito Residuo']],axis=1)
df2_bullet=df2_bullet[:-1]

In [115]:
index=['Quota Interesse','Interessi Totali','Debito Residuo']
df_bullet=pd.DataFrame(plan_bullet)
df_bullet.columns=index
df1_bullet=pd.concat([df,df_bullet],axis=1)
df2_bullet=pd.concat([df,df_bullet['Debito Residuo']],axis=1)
df2_bullet=df2_bullet[:-1]

**Output del Modello**

In [ ]:
if (amortization_type== 'Francese'):
  print (df2)
elif (amortization_type == 'Italiano'):
  print (df2_ita)
elif (amortization_type == 'Bullet'):
  print (df2_bullet)

In [56]:
market_data_path = '/content/drive/MyDrive/data.xlsx'
fixed_rates = pd.read_excel(market_data_path, sheet_name='fixings')
estimation_rate = pd.read_excel(market_data_path, sheet_name='estimation')
discount_rate = pd.read_excel(market_data_path, sheet_name='discount')

In [57]:
class IRS:

  def __init__(self, leg1_name = 'Receive', leg2_name = 'Pay', leg1_rate_type = 'Fixed', leg2_rate_type = 'Floating'):

    self.start_date = None
    self.leg1_flows = pd.DataFrame
    self.leg2_flows = pd.DataFrame
    self.leg1_name = leg1_name
    self.leg2_name = leg2_name
    self.leg1_rate_type = leg1_rate_type  
    self.leg2_rate_type = leg2_rate_type
    self.npv = None

In [58]:
def submit(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

  irs = IRS()

In [59]:
submit(path = market_data_path, underlying = underlying, amortization_type = amortization_type, notional = notional, amortization_rate = amortization_rate, maturity_label = maturity_label, ref_date = ref_date, tenor = tenor,
       business_convention = business_convention, date_gen_rule = date_gen_rule)

In [60]:
leg_fixed=FixedRateLeg(schedule,Actual360(),[notional],[0.05])
for i in leg_fixed:
  print(i.date(),i.amount())

July 27th, 2021 126.38888888888866
October 27th, 2021 127.77777777777777
January 27th, 2022 127.77777777777777
April 27th, 2022 124.99999999999956


prove npv obiettivo


In [62]:
int_list=list(map(int,df2['Debito Residuo']))
int_list

[10000, 7546, 5062, 2546]

In [84]:
df2['aaa']=df2['enddate']-df2['startdate']
df2['molt']=df2['Debito Residuo']*df2['aaa']
df2['tx']=['molt']*[0.0001]


TypeError: ignored